In [12]:
from optuna import Trial
from typing import Dict, Union, Any
from setfit import TrainingArguments
from setfit import SetFitModel

In [13]:
def model_init(params: Dict[str, Any]) -> SetFitModel:
    params = params or {}
    max_iter = params.get("max_iter", 100)
    solver = params.get("solver", "liblinear")
    params = {
        "head_params": {
            "max_iter": max_iter,
            "solver": solver,
        }
    }
    return SetFitModel.from_pretrained("KBLab/sentence-bert-swedish-cased", **params)


In [14]:
def hp_space(trial: Trial) -> Dict[str, Union[float, int, str]]:
    return {
        #"body_learning_rate": trial.suggest_float("body_learning_rate", 1e-6, 1e-3, log=True),
        "num_epochs": trial.suggest_int("num_epochs", 1, 10),
        "batch_size": trial.suggest_categorical("batch_size", [16, 32, 64]),
        #"seed": trial.suggest_int("seed", 1, 40),
        #"max_iter": trial.suggest_int("max_iter", 50, 300),
        #"solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear"]),
    }

In [15]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="../data/target_pilot_1_2_sv.csv")
dataset = dataset.class_encode_column("label")

/home/mun/miniconda3/envs/agreementclassifier/lib/python3.12/site-packages/datasets/utils/_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'text', 'label'],
        num_rows: 72
    })
})

In [17]:
dataset_tt = dataset["train"].train_test_split(train_size=58, stratify_by_column='label')

In [18]:
dataset_tt

DatasetDict({
    train: Dataset({
        features: ['ID', 'text', 'label'],
        num_rows: 58
    })
    test: Dataset({
        features: ['ID', 'text', 'label'],
        num_rows: 14
    })
})

In [19]:
from setfit import Trainer

In [20]:
args = TrainingArguments(
    batch_size=16,
    num_epochs=5,
)

In [21]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=dataset_tt["train"],
    eval_dataset=dataset_tt["test"]
)

/home/mun/miniconda3/envs/agreementclassifier/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Currently using DataParallel (DP) for multi-gpu training, while DistributedDataParallel (DDP) is recommended for faster training. See https://sbert.net/docs/sentence_transformer/training/distributed.html for more information.
/home/mun/miniconda3/envs/agreementclassifier/lib/python3.12/site-packages/datasets/utils/_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lin

In [22]:
trainer.train()
trainer.evaluate()

***** Running training *****
  Num unique pairs = 1758
  Batch size = 16
  Num epochs = 5


Step,Training Loss
1,0.301300
50,0.200900
100,0.002900
150,0.000600
200,0.000500
250,0.000400


***** Running evaluation *****                                                                                                                                                                                                                                                                                                                         


{'accuracy': 0.7142857142857143}

In [24]:
trainer.model.save_pretrained("../models/target_sv")

In [ ]:
best_run = trainer.hyperparameter_search(direction="maximize", hp_space=hp_space, n_trials=10)
print(best_run)

In [12]:
trainer.apply_hyperparameters(best_run.hyperparameters, final_model=True)
trainer.train()

/home/mun/miniconda3/envs/agreementclassifier/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 1218
  Batch size = 32
  Num epochs = 3


Step,Training Loss
1,0.281600
50,0.274900


In [13]:
trainer.evaluate()

***** Running evaluation *****


{'accuracy': 0.625}